In [1]:
import pandas as pd
import torch
from tqdm import tqdm
from sacrebleu.metrics import BLEU
from bert_score import score
from googletrans import Translator
import time
import numpy as np


d:\Data\HCMUS\Y4\Y4_S1\Nhap_mon_xu_ly_ngon_ngu_tu_nhien\Mid_term\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config
SRC = 'PDF1'
FROM_ID = 501
TO_ID = 625
INPUT_FILE = f"../output/alignment/{SRC}_{FROM_ID}_{TO_ID}.csv" 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def get_proxy_translations(df, sample_size):
    """
    Dịch câu nguồn tiếng Trung sang tiếng Việt dùng Google Translate
    để làm tham chiếu so sánh.
    """
    print(f"Đang lấy mẫu ngẫu nhiên {sample_size} cặp câu để đánh giá...")
    
    # Lấy mẫu ngẫu nhiên
    sample = df.sample(n=min(sample_size, len(df)), random_state=0).copy()

    translator = Translator()
    hypotheses = [] # Câu trong file 
    references = [] # Câu dịch máy (Làm chuẩn so sánh)
    success_count = 0
    
    for idx, row in tqdm(sample.iterrows(), total=len(sample)):
        src = str(row['src_lang'])
        tgt = str(row['tgt_lang']) # Đây là câu tiếng Việt gốc trong dataset
        
        # Bỏ qua câu quá ngắn hoặc rỗng
        if len(src) < 2 or len(tgt) < 5: 
            continue
            
        try:
            # Dịch Trung -> Việt
            translated = translator.translate(src, src='zh-cn', dest='vi').text
            
            # Câu Google dịch là Reference (Tham chiếu) và câu Tgt gốc là Hypothesis (Giả thuyết)
            references.append(translated) 
            hypotheses.append(tgt)
            
            success_count += 1
            time.sleep(0.1)
            
        except Exception as e:
            # Nếu lỗi mạng thì bỏ qua
            continue
            
    print(f"Đã dịch thành công {success_count} câu.")
    return hypotheses, references

def calculate_metrics(hypotheses, references):
    """Tính toán BLEU và BERTScore"""
    if not hypotheses:
        print("Không có dữ liệu để tính toán.")
        return

    # Tính BLEU Score
    # BLEU đánh giá độ khớp từ vựng (n-gram overlap)
    bleu = BLEU()
    bleu_score = bleu.corpus_score(hypotheses, [references])
    
    print(f"BLEU Score: {bleu_score.score:.2f}")

    # Tính BERTScore
    # BERTScore đánh giá độ tương đồng ngữ nghĩa (Semantic Similarity) dùng mô hình BERT
    P, R, F1 = score(hypotheses, references, lang='vi', verbose=False, device=DEVICE)
    
    avg_bert = F1.mean().item()
    print(f"BERTScore (F1): {avg_bert:.4f}")

    print("Ví dụ tham chiếu (Top 3):")
    for i in range(min(3, len(hypotheses))):
        print(f"Cặp {i+1}:")
        print(f"Dataset Tgt: {hypotheses[i]}")
        print(f"Google Trans: {references[i]}")

In [4]:
df = pd.read_csv(INPUT_FILE)
# Thực hiện quy trình đánh giá
SAMPLE_SIZE = int(df.shape[0] * 0.1)  # Số lượng câu lấy mẫu để đánh giá 
hyps, refs = get_proxy_translations(df, SAMPLE_SIZE)
# Tính điểm
calculate_metrics(hyps, refs)

Đang lấy mẫu ngẫu nhiên 14 cặp câu để đánh giá...


100%|██████████| 14/14 [00:05<00:00,  2.66it/s]


Đã dịch thành công 14 câu.
BLEU Score: 21.27
BERTScore (F1): 0.8252
Ví dụ tham chiếu (Top 3):
Cặp 1:
Dataset Tgt: Vì thế lời thề không thể đo được lòng chung thủy, không thể phán đoán đúng sai, nó chỉ có thể chứng minh rằng lúc nói ra lời thề cả hai người đều hết sức chân thành.
Google Trans: Vì vậy, những thứ như lời thề không thể đo lường được lòng chung thủy, cũng như không thể phán xét đúng hay sai. Chỉ có thể chứng minh rằng lúc này đã nói ra, đối phương đều là thật lòng.
Cặp 2:
Dataset Tgt: Bức thư viết cho chính mình số 509 Bạn không thế tiến vào trái tỉm một người nhiều khi không phải do bạn không cố gắng mà do người kia không ngừng gia cố bức tường bảo vệ trong trái tìm họ.
Google Trans: Thư 509 gửi bản thân Nếu bạn không thể đi vào trái tim người đó, không hẳn là do bạn làm việc chưa đủ chăm chỉ, mà là do người đó đang không ngừng gia cố bức tường trong trái tim mình.
Cặp 3:
Dataset Tgt: Bạn luôn nghĩ rằng tình cảm khiến bạn tổn thương thực ra người làm tổn thương bạn chính l